# test de foncitons 

In [53]:
import numpy as np
import msicpe.san as san
from plotly import express as px
import scipy.signal as signal
from matplotlib import pyplot as plt
import math

def distance_euclidienne(a, b):
    return round(math.sqrt(sum((x - y)**2 for x, y in zip(a, b))))


def Harmoniques(data, H):
    s = data['s']
    t = data['t']
    nu, S = san.trans_fourier(s,t)
    norm = np.log10(np.abs(S))
    #norm = logS/max(logS)
    #fig1 = px.line(x=nu, y=norm, title='log10 de TF de F(nu)', labels={'x':'nu', 'y':'log10(TF(Fx(t)))'}).show()
    fondamentaleTest = san.detect_fondamentale(norm, nu, 0.7)
    #print('fondamentaletest:', fondamentaleTest)
    fondamentale = san.detect_pics(norm, [120] ,nu) #on utilise la methode detect pics car la fonction detect fondamental ne fonctionne pas
    #print('ecart', fondamentale1-fondamentale)
    liste_freq = [i * fondamentale[0][0] for i in range(1,H+1)] #retourne une liste de frequences multiples de la fondamentale
    FreqHarmoniques, AmpliHarmoniques = san.detect_pics(norm, liste_freq, nu)
    return FreqHarmoniques, AmpliHarmoniques

def Moyennage(lettre, nb_harmoniques):
    '''cette fonction va nous permettre de faire la moyenne de chaque harmonique pour des signaux similaires. la lettre est un string'''
    freq=[]
    ampli=[]
    for i in range(1,6): #premet de créer une liste de l'emplacement où les sons sont stockés
        fichier = "data_partie_2/apprentissage/{}_{}.npz".format(lettre, i)
        data = np.load(fichier)
        F, A = Harmoniques(data, nb_harmoniques)
        freq.append(F)
        ampli.append(A)
    return np.mean(freq, axis=0), np.mean(ampli, axis=0)

nbH = 15
    
#insérer le son mystère ici
FreqMystere, AmplMystere = Harmoniques(np.load("data_partie_2/test/e_6.npz"),nbH)
#print("le son 3 a pour frequences",FreqMystere,"a pour amplitude" ,AmplMystere)

#plot amplitude en fonction de la frequece
'''plt.plot(Moyennage('a', nbH)[0], Moyennage('a', nbH)[1], 'blue') # bleu
plt.plot(Moyennage('e', nbH)[0], Moyennage('e', nbH)[1], 'orange') #orange
plt.plot(Moyennage('i', nbH)[0], Moyennage('i', nbH)[1], 'green') #vert
plt.plot(Moyennage('o', nbH)[0], Moyennage('o', nbH)[1], 'purple') #violet
plt.plot(Moyennage('u', nbH)[0], Moyennage('u', nbH)[1], 'brown') #marron
plt.plot(FreqMystere,AmplMystere, 'red') #rouge
plt.grid(axis='both')'''

for i in ['a','e', 'i', 'o', 'u']:
    #print('le son ',i ,' a pour harmoniques', Moyennage(i, nbH)[0], "et pour amplitude", Moyennage(i, nbH)[1])
    #print('FREQUENCES : disctance à',i ,' : ', distance_euclidienne(Moyennage(i, nbH)[0], FreqMystere))
    print('AMPLITUDES : disctance à',i ,' : ', distance_euclidienne(10*Moyennage(i, nbH)[1], 10*AmplMystere))


AMPLITUDES : disctance à a  :  17
AMPLITUDES : disctance à e  :  16
AMPLITUDES : disctance à i  :  55
AMPLITUDES : disctance à o  :  30
AMPLITUDES : disctance à u  :  50


## chtGPT

In [ ]:
def distance_euclidienne(a, b):
    return round(math.sqrt(sum((x - y)**2 for x, y in zip(a, b))))

def Harmoniques(data, H):
    s = data['s']
    t = data['t']
    nu, S = san.trans_fourier(s,t)
    logS = np.log10(np.abs(S))
    norm = logS/max(logS)
    #fig1 = px.line(x=nu, y=norm, title='log10 de TF de F(nu)', labels={'x':'nu', 'y':'log10(TF(Fx(t)))'}).show()
    fondamentaleTest = san.detect_fondamentale(norm, nu, 0.7)
    fondamentale = san.detect_pics(norm, [120] ,nu) #on utilise la methode detect pics car la fonction detect fondamental ne fonctionne pas
    liste_freq = [i * fondamentale[0][0] for i in range(1,H+1)] #retourne une liste de frequences multiples de la fondamentale
    FreqHarmoniques, AmpliHarmoniques = san.detect_pics(norm, liste_freq, nu)
    return FreqHarmoniques, AmpliHarmoniques

def Moyennage(lettre, nb_harmoniques):
    '''cette fonction va nous permettre de faire la moyenne de chaque harmonique pour des signaux similaires. la lettre est un string'''
    freq=[]
    ampli=[]
    for i in range(1,6): #premet de créer une liste de l'emplacement où les sons sont stockés
        fichier = "data_partie_2/apprentissage/{}_{}.npz".format(lettre, i)
        data = np.load(fichier)
        F, A = Harmoniques(data, nb_harmoniques)
        freq.append(F)
        ampli.append(A)
    return np.mean(freq, axis=0), np.mean(ampli, axis=0)

nbH = 15

#insérer le son mystère ici
FreqMystere, AmplMystere = Harmoniques(np.load("data_partie_2/etude/signal_2.npz"),nbH)
print("le son 3 a pour frequences",FreqMystere,"a pour amplitude" ,AmplMystere)

le son 3 a pour frequences [ 117.98033333  234.96083333  352.94116667  470.9215      587.902
  705.88233333  822.86283333  945.84233333 1057.82366667 1175.804
 1293.78433333 1418.7635     1525.74566667 1643.726      1771.70466667] a pour amplitude [0.95113105 1.         0.671564   0.48782855 0.17834003 0.3766166
 0.22960159 0.17469707 0.14136657 0.14519003 0.12786713 0.10072768
 0.12579224 0.11549147 0.16104862]


In [55]:
def distance_euclidienne(a, b):
    return round(math.sqrt(sum((x - y)**2 for x, y in zip(a, b))))

def corriger_fondamentale(norm, nu, H, f0_init=120, max_iter=8, tol=1e-3):
    """
    Corrige f0 en utilisant uniquement san.detect_pics().
    - norm : amplitude log du spectre
    - nu   : fréquences
    - H    : nombre d'harmoniques utilisées pour la correction
    """

    f0 = f0_init

    for it in range(max_iter):

        # --- 1. détecter la fondamentale ---
        det_f = san.detect_pics(norm, [f0], nu)

        try:
            f0_detect = float(det_f[0][0])
        except:
            # on essaie quand même d'avancer
            f0_detect = f0

        # --- 2. générer les cibles harmoniques ---
        liste_freq = [(k+1) * f0_detect for k in range(H)]

        det = san.detect_pics(norm, liste_freq, nu)

        # det = [ [freqs_detectées], [amps_detectées] ]
        freqs_detectees = np.array(det[0], dtype=float)

        # --- 3. recalculer une nouvelle estimation de f0 ---
        ratios = freqs_detectees / np.arange(1, H+1)

        f0_new = np.median(ratios)

        # --- 4. vérifier convergence ---
        if abs(f0_new - f0) < tol:
            return f0_new

        f0 = f0_new

    return f0  # dernière estimation si non convergent

def detecter_harmoniques(norm, nu, H, f0_init=120):

    # corriger la fondamentale
    f0 = corriger_fondamentale(norm, nu, H, f0_init=f0_init)

    # liste harmonique finale
    liste_freq = [(k+1)*f0 for k in range(H)]

    # détection finale des pics
    FreqHarmoniques, AmpliHarmoniques = san.detect_pics(norm, liste_freq, nu)

    return f0, np.array(FreqHarmoniques, float), np.array(AmpliHarmoniques, float)


def Harmoniques(data, H):
    s = data['s']
    t = data['t']
    nu, S = san.trans_fourier(s,t)
    norm = np.log10(np.abs(S))
    #logS = logS/max(logS)
    fig1 = px.line(x=nu, y=norm, title='log10 de TF de F(nu)', labels={'x':'nu', 'y':'log10(TF(Fx(t)))'}).show()
    fondamentaleTest = san.detect_fondamentale(norm, nu, 0.7)
    print("fondTEst:", fondamentaleTest)
    fondamentale = san.detect_pics(norm, [120] ,nu) #on utilise la methode detect pics car la fonction detect fondamental ne fonctionne pas
    liste_freq = [i * fondamentale[0][0] for i in range(1,H+1)] #retourne une liste de frequences multiples de la fondamentale
    FreqHarmoniques, AmpliHarmoniques = san.detect_pics(norm, liste_freq, nu)
    return FreqHarmoniques, AmpliHarmoniques

def Moyennage(lettre, nb_harmoniques):
    '''cette fonction va nous permettre de faire la moyenne de chaque harmonique pour des signaux similaires. la lettre est un string'''
    freq=[]
    ampli=[]
    for i in range(1,6): #premet de créer une liste de l'emplacement où les sons sont stockés
        fichier = "data_partie_2/apprentissage/{}_{}.npz".format(lettre, i)
        data = np.load(fichier)
        F, A = Harmoniques(data, nb_harmoniques)
        freq.append(F)
        ampli.append(A)
    return np.mean(freq, axis=0), np.mean(ampli, axis=0)

nbH = 15

print(Harmoniques(np.load("data_partie_2/apprentissage/a_1.npz"),15))



fondTEst: 0.0
(array([ 106.98216667,  214.96416667,  321.94633333,  429.92833333,
        539.91      ,  655.89066667,  762.87283333,  869.855     ,
        990.83483333, 1089.81833333, 1207.79866667, 1314.78083333,
       1437.76033333, 1532.7445    , 1654.72416667]), array([2.6437879, 2.5499532, 1.9889917, 1.9406531, 1.4740192, 1.2831967,
       1.7164677, 1.8667094, 1.3088956, 1.2105188, 1.415637 , 1.6295398,
       1.1177787, 0.7847352, 0.6700628], dtype=float32))
